**Yixuan Qiu**

Spring 2020

CS 251: Data Analysis and Visualization

Project 6: Supervised learning

In [2]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=5)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Task 3: Naive Bayes Classifier

After finishing your email preprocessing pipeline, implement the one other supervised learning algorithm we we will use to classify email, **Naive Bayes**.

### 3a) Implement Naive Bayes

In `naive_bayes.py`, implement the following methods:
- Constructor
- `train(data, y)`: Train the Naive Bayes classifier so that it records the "statistics" of the training set: class priors (i.e. how likely an email is in the training set to be spam or ham?) and the class likelihoods (the probability of a word appearing in each class — spam or ham).
- `predict(data)`: Combine the class likelihoods and priors to compute the posterior distribution. The predicted class for a test sample is the class that yields the highest posterior probability.
- `accuracy(y, y_pred)`: The usual definition :)


#### Bayes rule ingredients: Priors and likelihood (`train`)

To compute class predictions (probability that a test example belong to either spam or ham classes), we need to evaluate **Bayes Rule**. This means computing the priors and likelihoods based on the training data.

**Prior:** $$P_c = \frac{N_c}{N}$$ where $P_c$ is the prior for class $c$ (spam or ham), $N_c$ is the number of training samples that belong to class $c$ and $N$ is the total number of training samples.

**Likelihood:** $$L_{c,w} = \frac{N_{c,w} + 1}{N_{c} + M}$$ where
- $L_{c,w}$ is the likelihood that word $w$ belongs to class $c$ (*i.e. what we are solving for*)
- $N_{c,w}$ is the total count of **word $w$** in emails that are only in class $c$ (*either spam or ham*)
- $N_{c}$ is the total number of **all words** that appear in emails of the class $c$ (*total number of words in all spam emails or total number of words in all ham emails*)
- $M$ is the number of features (*number of top words*).

#### Bayes rule ingredients: Posterior (`predict`)

To make predictions, we now combine the prior and likelihood to get the posterior:

**Posterior:** $$\text{Post}_{i, c} = Log(P_c) + \sum_{j \in J_i}Log(L_{c,j})$$ where
- $\text{Post}_c$ is the posterior for class $c$ for test sample $i$(*i.e. evidence that email $i$ is spam or ham*). What we are solving for.
- $Log(P_c)$ is the logarithm of the prior for class $c$ $P_c$.
- $j \in J_i$ (under the sum) indexes the set of words in the current test sample that have nonzero counts (*i.e. which words show up in the current test set email $i$? $j$ is the index of each of these words.*)
- $\sum_{j \in J_i}Log(L_{c,j})$: we sum over the log-likelihoods ONLY PERTAINING TO CLASS $c$ at word word indices that appear in the current test email $i$ (i.e. indices at which the counts are > 0).

In [3]:
from naive_bayes_multinomial import NaiveBayes

#### Test `train`

In [4]:
num_test_classes = 4
np.random.seed(0)
data_test = np.random.random(size=(100, 6))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_test, y_test)

print(f'Your class priors are: {nbc.class_priors}\nand should be          [0.24 0.26 0.25 0.25].')
print(f'Your class likelihoods shape is {nbc.class_likelihoods.shape} and should be (4, 6).')
print(f'Your likelihoods are:\n{nbc.class_likelihoods}')


test_likelihoods = np.array([[0.15116, 0.18497, 0.17571, 0.1463 , 0.16813, 0.17374],
       [0.16695, 0.17437, 0.15742, 0.16887, 0.15677, 0.17562],
       [0.14116, 0.1562 , 0.19651, 0.17046, 0.17951, 0.15617],
       [0.18677, 0.18231, 0.15884, 0.12265, 0.16755, 0.18187]])
print(f'and should be\n{test_likelihoods}')

Your class priors are: [0.24 0.26 0.25 0.25]
and should be          [0.24 0.26 0.25 0.25].
Your class likelihoods shape is (4, 6) and should be (4, 6).
Your likelihoods are:
[[0.15116 0.18497 0.17571 0.1463  0.16813 0.17374]
 [0.16695 0.17437 0.15742 0.16887 0.15677 0.17562]
 [0.14116 0.1562  0.19651 0.17046 0.17951 0.15617]
 [0.18677 0.18231 0.15884 0.12265 0.16755 0.18187]]
and should be
[[0.15116 0.18497 0.17571 0.1463  0.16813 0.17374]
 [0.16695 0.17437 0.15742 0.16887 0.15677 0.17562]
 [0.14116 0.1562  0.19651 0.17046 0.17951 0.15617]
 [0.18677 0.18231 0.15884 0.12265 0.16755 0.18187]]


#### Test `predict`

In [5]:
num_test_classes = 4
np.random.seed(0)
data_train = np.random.random(size=(100, 10))
data_test = np.random.random(size=(4, 10))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_train, y_test)
test_y_pred = nbc.predict(data_test)

print(f'Your predicted classes are {test_y_pred} and should be [2 2 2 2].')

Your predicted classes are [2 2 2 2] and should be [2 2 2 2].


### 3c) Spam filtering

Let's start classifying spam email using the Naive Bayes classifier.

- Use `np.load` to load in the train/test split that you created last week.
- Use your Naive Bayes classifier on the Enron email dataset!

**Question 9:** What accuracy do you get on the test set with Naive Bayes?

**Answer 9:**<br>
0.89

In [6]:
import email_preprocessor as epp

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qiuyixuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Load your training and test data into numpy ndarrays using np.load()
# (the files you created at the end of the previous notebook)

train_data = np.load('data/email_train_x.npy')
test_data = np.load('data/email_test_x.npy')
y_train = np.load('data/email_train_y.npy')
y_test = np.load('data/email_test_y.npy')

In [8]:
# Construct your classifier
nb = NaiveBayes(2)

In [9]:
# Train and test your classifier
nb.train(train_data, y_train)
y_pred = nb.predict(test_data)
accuracy = nb.accuracy(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8902077151335311


### 3d) Confusion matrix

To get a better sense of the errors that the Naive Bayes classifer makes, you will create a confusion matrix. 

- Implement `confusion_matrix` in `naive_bayes.py`.
- Print out a confusion matrix of the spam classification results.

In [10]:
print(nb.confusion_matrix(y_test, y_pred))

[[3237  175]
 [ 565 2763]]


**Question 10:** Interpret the confusion matrix, using the convention that positive detection means spam (*e.g. a false positive means classifying a ham email as spam*). What types of errors are made more frequently by the classifier? What does this mean (*i.e. X (spam/ham) is more likely to be classified than Y (spam/ham) than the other way around*)?

**Reminder: Look back at your preprocessing code: which class indices correspond to spam/ham?**

**Answer 10:**<br>
False positive errors.<br>
A ham is more likely to be classified as spam than the other way around.

### 3e) Investigate the misclassification errors

Numbers are nice, but they may not the best for developing your intuition. Sometimes, you want to see what an misclassification *actually* looks like to build your understanding as you look to improve your algorithm. Here, you will take a false positive and a false negative misclassification and retrieve the actual text of the email so see which emails produced the error.

- Determine the index of the **FIRST** false positive and false negative misclassification — i.e. 2 indices in total. Remember to use your inds array to figure out the index of the emails BEFORE shuffling happened.
- **Section B:** Implement the function `retrieve_emails` in `email_preprocessor.py` to return the string of the raw email at the error indices. (**Sections A/C** have been supplied with this function on Classroom.)
- Call your function to print out the two emails that produced misclassifications.

**Question 11:** What do you think it is about each email that resulted in it being misclassified?

**Answer 11:**<br>
The first email is long, so it has a higher chance to hit the words common in spams.<br>
The second email is very short with weird words. It does not contain enough useful information for us to predict, so it is likely to be classified as a spam.

In [11]:
# Determine the indices of the 1st FP and FN.
# Note: spam = 0, ham = 1

fn = np.where(np.logical_and(y_test==0, y_pred==1))[0][0]
fp = np.where(np.logical_and(y_test==1, y_pred==0))[0][0]

inds_test = np.load('data/email_test_inds.npy')
fn = inds_test[fn]
fp = inds_test[fp]

In [12]:
print("fp =", fp)
print("fn =", fn)

fp = 24632
fn = 8258


In [13]:
# Use retrieve_emails() to display the first FP and FN.
inds = np.array([fp, fn])
emails = epp.retrieve_emails(inds)

print()
print('The 1st email that is a false positive (classified as spam, but really not) is:')
print('------------------------------------------------------------------------------------------')
print(emails[0])
print('------------------------------------------------------------------------------------------')
print('The 1st email that is a false negative (classified as ham, but really spam) is:')
print('------------------------------------------------------------------------------------------')
print(emails[1])
print('------------------------------------------------------------------------------------------')


The 1st email that is a false positive (classified as spam, but really not) is:
------------------------------------------------------------------------------------------
Subject: will be on the front page of google in just 48 hours ! guaranteed !
your bed and breakfast will be on
the front page of google
in only 48 hours
or your money back ! guaranteed !
as partners with
google . com and the google network we can offer to place your web site on the
front page for your listing !
this means that over 80 million real
userswho use the google search engineand the google network every
day can see your bed and breakfast web site !
all for only
£ 79 . 95 for six months
how does it work ?
everyone that types in
any of these most used phrases below along with your town or city will see your
site come up on the first page ! guaranteed !
for example
if you are a bed and breakfastin manchesterpeople looking for a bb
in manchesterwould type in the search boxbed and
breakfast inmanchester .
below a

## Task 4) Comparison with KNN


- Run a similar analysis to what you did with Naive Bayes above. When computing accuracy on the test set, you may want to reduce the size of the test set (e.g. to the first 500 emails in the test set).
- Copy-paste your `confusion_matrix` method into `knn.py` so that you can run the same analysis on a KNN classifier.

In [14]:
from knn import KNN

In [15]:
# Construct and train your KNN classifier
classifier = KNN(2)
classifier.train(train_data, y_train)

In [16]:
# Evaluate the accuracy of the KNN classifier
num_test = 500
y_pred = classifier.predict(test_data[:num_test], 2)
accuracy = classifier.accuracy(y_test[:num_test], y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.898


In [17]:
print(classifier.confusion_matrix(y=y_test[:num_test], y_pred=y_pred))

[[253  11]
 [ 40 196]]


**Question 12:** What accuracy did you get on the test set (potentially reduced in size)?

**Answer 12:**<br>
0.898

**Question 13:** How does the confusion matrix compare to that obtained by Naive Bayes?

**Answer 13:**<br>
False positive errors are still more frequently made by the classifier.<br>
A ham is more likely to be classified as spam than the other way around.<br>
The false positive rate of the Naive Bayes classifier is 17.0%, and the false positive rate of the KNN classifier is 16.9%.<br>
The true positive rate of the Naive Bayes classifier is 94.9%, and the true positive rate of the KNN classifier is 95.8%.<br>
It seems that the result obtained by KNN is more accurate.

**Question 14:** Briefly describe at least one pro/con of KNN compared to Naive Bayes on this dataset.

**Answer 14:**<br>
pro of KNN: slightly more accurate--a higher true positive rate and a lower false positive rate.<br>
con of KNN: slower than Naive Bayes.<br>

**Question 15:** When potentially reducing the size of the test set here, why is it important that we shuffled our train and test set?

**Answer 15:**<br>
To avoid highly correlated data samples within a small set.

## Extensions (Section B only)

### 1. Better text preprocessing

- If you look at the top words extracted from the email dataset, many of them are common "stop words" (e.g. a, the, to, etc.) that do not carry much meaning when it comes to differentiating between spam vs. non-spam email. Improve your preprocessing pipeline by building your top words without stop words. Analyze performance differences.

  
I made a `remove_stop_words` method in email_preprocessor using NLTK.<br>
I added an optional boolean parameter--`remove_sw` whose default value is False--in count_words() and make_feature_vectors() in email_preprocessor.

#### Determine the count of each word in the dataset, ignoring stop words

In [18]:
word_freq, num_emails = epp.count_words(remove_sw=True)

#### Compile a list of the top `num_features` words and their respective counts.

In [19]:
top_words, top_counts = epp.find_top_words(word_freq)
print(f"Your top 5 words are\n{top_words[:5]}")
print(f"The associated counts are\n{top_counts[:5]}")

Your top 5 words are
['enron', 'subject', 'ect', 'com', 'company']
The associated counts are
[60909, 47811, 35346, 24185, 22959]


The output shows that stop words like 'the' 'to' 'and' 'of' 'a' are removed.

#### Make a feature vector of counts

In [20]:
features, y = epp.make_feature_vectors(top_words, num_emails, remove_sw=True)

In [21]:
np.random.seed(0)
x_train_nosw, y_train_nosw, inds_train_nosw, x_test_nosw, y_test_nosw, inds_test_nosw = epp.make_train_test_sets(features, y)

In [22]:
np.save('data/email_train_x_nosw.npy', x_train_nosw)
np.save('data/email_train_y_nosw.npy', y_train_nosw)
np.save('data/email_train_inds_nosw.npy', inds_train_nosw)
np.save('data/email_test_x_nosw.npy', x_test_nosw)
np.save('data/email_test_y_nosw.npy', y_test_nosw)
np.save('data/email_test_inds_nosw.npy', inds_test_nosw)

In [23]:
train_data_nosw = np.load('data/email_train_x_nosw.npy')
test_data_nosw = np.load('data/email_test_x_nosw.npy')
y_train_nosw = np.load('data/email_train_y_nosw.npy')
y_test_nosw = np.load('data/email_test_y_nosw.npy')

In [24]:
nb_nosw = NaiveBayes(2)

#### Train and test classifier

In [25]:
nb_nosw.train(train_data_nosw, y_train_nosw)
y_pred_nosw = nb_nosw.predict(test_data_nosw)
accuracy_nosw = nb_nosw.accuracy(y_test_nosw, y_pred_nosw)
print("Accuracy without stop words:", accuracy_nosw)

Accuracy without stop words: 0.9277448071216617


The accuracy of the classifier is much higher after the stop words are removed.

In [26]:
print("Confusion Matrix without stop words:\n", nb_nosw.confusion_matrix(y_test_nosw, y_pred_nosw))

Confusion Matrix without stop words:
 [[3298  114]
 [ 373 2955]]


The condusion matrix shows that:<br>
False positive errors (ham classified as spam) are still more frequently made by the classifier.<br>
The false positive rate becomes lower and is 11.2%.<br>
The true positive rate is 96.7%, higher than with stop words.<br>
These stats indicate the improved performance of the classifier with stop words removed.

### 2. Feature size

- Explore how the number of selected features for the email dataset influences accuracy and runtime performance.

I added an optional parameter `num_features` with a default value 200 in `make_feature_vectors` of email_preprocessor.

In [27]:
import time

#### Feature Size = 100

In [28]:
features, y = epp.make_feature_vectors(top_words, num_emails, num_features = 100)

In [29]:
np.random.seed(0)
x_train_100, y_train_100, inds_train_100, x_test_100, y_test_100, inds_test_100 = epp.make_train_test_sets(features, y)
np.save('data/email_train_x_100.npy', x_train_100)
np.save('data/email_train_y_100.npy', y_train_100)
np.save('data/email_train_inds_100.npy', inds_train_100)
np.save('data/email_test_x_100.npy', x_test_100)
np.save('data/email_test_y_100.npy', y_test_100)
np.save('data/email_test_inds_100.npy', inds_test_100)

In [30]:
train_data_100 = np.load('data/email_train_x_100.npy')
test_data_100 = np.load('data/email_test_x_100.npy')
y_train_100 = np.load('data/email_train_y_100.npy')
y_test_100 = np.load('data/email_test_y_100.npy')

In [31]:
nb_100 = NaiveBayes(2)

In [32]:
start = time.time()
nb_100.train(train_data_100, y_train_100)
y_pred_100 = nb_100.predict(test_data_100)
end = time.time()
print("Feature Size = 100\nTime for training and predicting:", end - start)
accuracy_100 = nb_100.accuracy(y_test_100, y_pred_100)
print("Accuracy:", accuracy_100)
print("Confusion Matrix:\n", nb_100.confusion_matrix(y_test_100, y_pred_100))

Feature Size = 100
Time for training and predicting: 0.15915918350219727
Accuracy: 0.886646884272997
Confusion Matrix:
 [[3275  137]
 [ 627 2701]]


#### Feature size = 200

Accuracy: 0.890<br>
Confusion Matrix:<br>
[[3237  175]<br>
 [ 565 2763]]

#### Feature Size = 300

In [33]:
features, y = epp.make_feature_vectors(top_words, num_emails, num_features = 300)

In [34]:
np.random.seed(0)
x_train_300, y_train_300, inds_train_300, x_test_300, y_test_300, inds_test_300 = epp.make_train_test_sets(features, y)
np.save('data/email_train_x_300.npy', x_train_300)
np.save('data/email_train_y_300.npy', y_train_300)
np.save('data/email_train_inds_300.npy', inds_train_300)
np.save('data/email_test_x_300.npy', x_test_300)
np.save('data/email_test_y_300.npy', y_test_300)
np.save('data/email_test_inds_300.npy', inds_test_300)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qiuyixuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
train_data_300 = np.load('data/email_train_x_300.npy')
test_data_300 = np.load('data/email_test_x_300.npy')
y_train_300 = np.load('data/email_train_y_300.npy')
y_test_300 = np.load('data/email_test_y_300.npy')

In [36]:
nb_300 = NaiveBayes(2)

In [37]:
start = time.time()
nb_300.train(train_data_300, y_train_300)
y_pred_300 = nb_300.predict(test_data_300)
end = time.time()
print("Feature Size = 300\nTime for training and predicting:", end - start)
accuracy_300 = nb_300.accuracy(y_test_300, y_pred_300)
print("Accuracy:", accuracy_300)
print("Confusion Matrix:\n", nb_300.confusion_matrix(y_test_300, y_pred_300))

Feature Size = 300
Time for training and predicting: 0.21155190467834473
Accuracy: 0.937833827893175
Confusion Matrix:
 [[3286  126]
 [ 293 3035]]


There is always a **speed-accuracy trade-off**.<br>

It takes more time to train and predict datasets when the feature size is larger.<br>

The accuracy gets improved as the feature size increases.<br>

##### Compare the confusion matrices:<br>

When feature size = 100,<br>
the false positive rate is 18.8%, and the true positive rate is 96.0%.<br>

When feature size = 200,<br>
the false positive rate is 17.0%, and the true positive rate is 94.9%.<br>

When feature size = 300,<br>
the false positive rate is 8.8%, and the true positive rate is 96.3%.<br>

The classifier performs the best when the feature size is 300.<br>
However, when the feature size is 100, it has a higher TPR; when the feature size is 300, it has a lower FPR.<br> So they both have some advantages over each other.

### 3. Distance metrics
- Compare KNN performance with the $L^2$ and $L^1$ distance metrics

I added an optional parameter `dist_metrics` in KNN's `predict()` that allows me to choose different distance metrics.

In [38]:
num_test = 500

#### L2 -- Euclidean Distance

In [43]:
knn_l2 = KNN(2)
start = time.time()
knn_l2.train(train_data, y_train)
y_pred_l2 = knn_l2.predict(test_data[:num_test], 2, dist_metrics='L2')
end = time.time()
accuracy_l2 = knn_l2.accuracy(y_test[:num_test], y_pred_l2)
print("L2 performance\nAccuracy:", accuracy_l2)
print("Confusion Matrix:\n", knn_l2.confusion_matrix(y=y_test[:num_test], y_pred=y_pred_l2))
print("Time for training and predicting:", end - start)

L2 performance
Accuracy: 0.898
Confusion Matrix:
 [[253  11]
 [ 40 196]]
Time for training and predicting: 208.33208179473877


#### L1 -- Manhattan Distance

I used SciPy's `cityblock` method to compute the Manhattan distances between data samples.

In [44]:
knn_l1 = KNN(2)
start = time.time()
knn_l1.train(train_data, y_train)
y_pred_l1 = knn_l1.predict(test_data[:num_test], 2, dist_metrics='L1')
end = time.time()
accuracy_l1 = knn_l1.accuracy(y_test[:num_test], y_pred_l1)
print("L1 performance\nAccuracy:", accuracy_l1)
print("Confusion Matrix:\n", knn_l1.confusion_matrix(y=y_test[:num_test], y_pred=y_pred_l1))
print("Time for training and predicting:", end - start)

L1 performance
Accuracy: 0.894
Confusion Matrix:
 [[254  10]
 [ 43 193]]
Time for training and predicting: 230.47265696525574


The results are very similar.<br>
Using the L2 distance metric generates a slightly higher accuracy.

For both L1 and L2, false positive errors are more frequent.<br>
The false positive rate with L1 is 18.2%, and that with L2 if 16.9%.<br>
L2 is better at minimizing errors as it predicts fewer false positives.<br>
The true positive rate with L1 is 96.2%, and that with L2 if 95.8%.<br>
L1 is better at detecting valid solutions/true positives, which makes sense as the L1-norm is diamond-shaped.<br>

It took the classifier 230 seconds to train and predict with the L1 distance metrix.<br>
With L2, it took 208 seconds.<br>
So, in this case, L2 is slightly faster.

Since I limited the number of features to 500 which is small, KNN performs slightly better with L2 than with L1.<br>
For higher dimensional data, L1 may win.

### 4. K-Fold Cross-Validation

### Credit:

Oliver

Hannah

https://stackoverflow.com/questions/47736531/vectorized-matrix-manhattan-distance-in-numpy